# Consigna ✍️

Eres parte del equipo de datos de PaperGalaxy S.A., una empresa especializada en la venta de artículos de oficina. Tu jefa, Mariana Ruiz, te ha solicitado un análisis exhaustivo sobre las ventas, clientes y jerarquía del equipo comercial. Contarás con la base de datos sales_office.db, que contiene información histórica de ventas, empleados y clientes.

Deberás utilizar subconsultas, CTEs (simples y recursivas), y funciones de ventana para responder diferentes preguntas clave del negocio.

## HW1 - Subconsultas

### 1. Productos con precio mayor al promedio general

Usar una subconsulta no correlacionada para listar los productos con un precio mayor al promedio general de todos los productos vendidos.

In [1]:
import pandas as pd
import sqlite3
import db_methods

query = """
SELECT * FROM sales
WHERE priceeach > (
    SELECT AVG(priceeach) FROM sales
);
"""
with sqlite3.connect("sales_office.db") as db_connection:
    data = pd.read_sql_query(query, db_connection)
    
data.head()

,id,productcode,productname,productline,priceeach,quantityordered,ordernumber,sales,year_id,month_id
0,3,P001,Lapicera Azul,Papelería,299.60,4,ORD394yT,1198.40,2024,5
1,5,P004,Monitor 24'',Tecnología,229.69,18,ORD513Jx,4134.42,2023,7
2,6,P001,Lapicera Azul,Papelería,159.60,15,ORD078CX,2394.00,2023,3
3,8,P004,Monitor 24'',Tecnología,233.30,14,ORD427tv,3266.20,2024,6
4,10,P007,Set Resaltadores,Papelería,236.35,8,ORD576Pr,1890.80,2023,3


### 2. Productos con precio mayor al promedio de la linea de producto
Utilizar una subconsulta correlacionada para encontrar los productos cuyo precio es mayor al promedio de su línea de producto (PRODUCTLINE).

In [ ]:

with sqlite3.connect("sales_office.db") as db_connection:
    query = """
        SELECT 
            productline,
            year_id,
            AVG(priceeach) AS avg_price
        FROM sales
        GROUP BY productline, year_id;
    """
    data = pd.read_sql_query(query, db_connection)
    print(len(data))

data

In [ ]:

with sqlite3.connect("sales_office.db") as db_connection:
    query = """
        WITH avg_sales AS (
            SELECT 
                productline,
                AVG(priceeach) AS avg_price
            FROM sales
            GROUP BY productline
        )
        SELECT 
            s.productline,
            s.productname,
            s.priceeach
        FROM sales s
        JOIN avg_sales a ON s.productline = a.productline
        WHERE s.priceeach > a.avg_price;
    """
    data = pd.read_sql_query(query, db_connection)
    print(len(data))

data

In [ ]:
with sqlite3.connect("sales_office.db") as db_connection:
    query = """
WITH AvgPricePerLineYear AS (
    SELECT 
        productline,
        year_id,
        AVG(priceeach) AS avg_price
    FROM sales
    GROUP BY productline, year_id
),
AboveAverageProducts AS (
    SELECT 
        s.*,
        avg.avg_price,
        avg.year_id AS avg_year,
        avg.productline AS avg_productline
    FROM sales s
    JOIN AvgPricePerLineYear avg
        ON s.productline = avg.productline
        AND s.year_id = avg.year_id
    WHERE s.priceeach > avg.avg_price
),
RankedProducts AS (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY year_id, productline 
               ORDER BY priceeach DESC
           ) AS rn
    FROM AboveAverageProducts
)
SELECT *
FROM RankedProducts
WHERE rn = 1;

    """
    data = pd.read_sql_query(query, db_connection)
    print(len(data))

data

In [ ]:
with sqlite3.connect("sales_office.db") as db_connection:
    query = """
    -- obtener los codigos unicos de producto
    SELECT DISTINCT productcode
    FROM sales;
    """
    data = pd.read_sql_query(query, db_connection)
    print(len(data))
data

## HW2 - CTEs
### 1. Reescribir la consulta del punto 1.b usando una CTE en lugar de subconsulta correlacionada.


In [2]:
query = """
WITH avg_price AS (
    SELECT AVG(priceeach) AS avg_priceeach
    FROM sales
)
SELECT *
FROM sales, avg_price
WHERE sales.priceeach > avg_price.avg_priceeach;
"""

with sqlite3.connect("sales_office.db") as db_connection:
    data = pd.read_sql_query(query, db_connection)

data.head()

,id,productcode,productname,productline,priceeach,quantityordered,ordernumber,sales,year_id,month_id,avg_priceeach
0,3,P001,Lapicera Azul,Papelería,299.60,4,ORD394yT,1198.40,2024,5,146.0316
1,5,P004,Monitor 24'',Tecnología,229.69,18,ORD513Jx,4134.42,2023,7,146.0316
2,6,P001,Lapicera Azul,Papelería,159.60,15,ORD078CX,2394.00,2023,3,146.0316
3,8,P004,Monitor 24'',Tecnología,233.30,14,ORD427tv,3266.20,2024,6,146.0316
4,10,P007,Set Resaltadores,Papelería,236.35,8,ORD576Pr,1890.80,2023,3,146.0316


## 2. Calcular el porcentaje de clientes cuyo campo state es NULL en la tabla customers.


In [3]:
with sqlite3.connect("sales_office.db") as db_connection:
    query="""
    WITH counts AS (
    SELECT 
        COUNT(*) AS total_rows,
        COUNT(state) AS non_null_states
    FROM customers
)
SELECT 
    100.0 * (total_rows - non_null_states) / total_rows AS null_state_percentage
FROM counts;
    """
    data = pd.read_sql_query(query, db_connection)

data

,null_state_percentage
0,12.0


## 3. Obtener todos los empleados que reportan directa o indirectamente a Mariana (id = 4) y Ricardo (id = 7). Asegúrate de incluir un campo root_id para indicar a cuál jefe se conecta cada empleado.

In [ ]:
import pandas as pd
import sqlite3
import db_methods

with sqlite3.connect("sales_office.db") as db_connection:
    query="""
    WITH RECURSIVE hierarchy AS (
    -- Nivel base: Mariana y Ricardo
    SELECT 
        id AS root_id,
        id,
        name,
        boss_id
    FROM employees
    WHERE id IN (4, 7)

    UNION ALL

    -- Paso recursivo: encontrar subordinados
    SELECT 
        h.root_id,
        e.id,
        e.name,
        e.boss_id
    FROM employees e
    JOIN hierarchy h ON e.boss_id = h.id
    )
    SELECT *
    FROM hierarchy
    --WHERE id != root_id;  -- excluir a los jefes mismos si quieres solo a sus reportes
    """
    data = pd.read_sql_query(query, db_connection)
    len(data)
    data

## 4. 
* Aplicar ROW_NUMBER() para identificar registros duplicados en la tabla sales, considerando ORDERNUMBER, PRODUCTCODE, y QUANTITYORDERED.


In [ ]:
with sqlite3.connect("sales_office.db") as db_connection:
    # Aplicar ROW_NUMBER() para identificar registros duplicados en la tabla sales, considerando ORDERNUMBER, PRODUCTCODE, y QUANTITYORDERED.

    query = """
    WITH RankedSales AS (
        SELECT 
            *,
            ROW_NUMBER() OVER (
                PARTITION BY ordernumber, productcode, quantityordered 
                ORDER BY ordernumber, productcode, quantityordered
            ) AS rn
        FROM sales
    )
    """
    data = pd.read_sql_query(query, db_connection)
    print(len(data))
data

## 4. 
* Usar RANK() y DENSE_RANK() para generar un ranking de ventas (sales) por año (YEAR_ID), ordenadas de mayor a menor.


In [ ]:
with sqlite3.connect("sales_office.db") as db_connection:
    query = """
    WITH sales_by_year AS (
        SELECT 
            year_id,
            SUM(priceeach) AS total_sales
        FROM sales
        GROUP BY year_id
    )
    SELECT
    year_id,
    total_sales,
    RANK() OVER (
        ORDER BY total_sales DESC
    ) AS sales_rank
    """
    data = pd.read_sql_query(query, db_connection)
    print(len(data))
data

* Usar LAG() para calcular la variación porcentual de ventas por mes respecto al mes anterior, dentro de cada año.


In [ ]:
with sqlite3.connect("sales_office.db") as db_connection:
    query = """
    WITH sales_by_year AS (
        SELECT 
            year_id,
            month_id,
            SUM(priceeach) AS total_sales
        FROM sales
        GROUP BY year_id, month_id
        )
    SELECT
        year_id,
        month_id,
        total_sales,
        LAG() OVER (
            PARTITION BY year_id 
            ORDER BY month_id
        ) AS previous_month_sales
    FROM sales_by_year
    WHERE previous_month_sales IS NOT NULL
    ORDER BY year_id, month_id;
    """
    data = pd.read_sql_query(query, db_connection)
    print(len(data))
data



* Usar LEAD() para mostrar las ventas del mes siguiente junto a las actuales.